In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import mysql.connector as con
from datetime import datetime
from transformers import pipeline
import re

In [6]:
summarizer= pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [7]:
db= con.connect(host='localhost', user='root', password='@Amir123', database='college')
cursor= db.cursor()

In [8]:
data_query="SELECT MEMBER_ID, DOC_TITL, DOC_DESC, DOC_SDATE,portalid FROM kf_docmnt where DOC_PUBDATE='2023-03-10'"
cursor.execute(data_query)
news_data= cursor.fetchall()

In [9]:
len(news_data)

138

In [10]:
news_summary_1702=[]
for i in range(len(news_data)):
    summary=summarizer(news_data[i][2],min_length=100, do_sample=False)
    news_summary_1702.append((news_data[i][0],summary))
    print(i,"out of",len(news_data),"have been summarized")

0 out of 138 have been summarized
1 out of 138 have been summarized


KeyboardInterrupt: 

In [6]:
for i in range(len(news_data)):
    final_dataset=(news_data[i][4],news_data[i][1],news_data[i][2],news_summary_1702[i][1][0]['summary_text'],news_data[i][0],news_data[i][3])
    insert_sum= "INSERT INTO NEWS_SUMMARY VALUES(%s,%s,%s,%s,%s,%s,'News','News','2023-03-09')"
    cursor.execute(insert_sum, final_dataset)
    db.commit()